### Configuration
- Vector Store : ChromaDB
- QA model : Ollama.LLAMA3.2
- Embedding model : Nomic Embed text
- Split method : Langchain.TokenTextSplitter CS : 4900 Overlap 200 previously 10000

### Imports

In [ ]:
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings
from langchain_ollama.llms import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate

from tqdm import tqdm
from rag.utils.my_indexing_utils import *
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate


#### Set parameters


In [44]:
CHUNK_SIZE = 2500
OVERLAP = 200
EMBEDDING_MODEL="nomic-embed-text"
DOCUMENTS_DIR="/Users/mtis/Local/Code/GitRepos/LangAI/data/raw_data"
SPLITTED_DOCS_DIR="/Users/mtis/Local/Code/GitRepos/LangAI/rag/splitted_docs"
PERSISTENT_DIR="data/chroma_langchain_db"
SAMPLE_SIZE=1
VERBOSE=True
BATCH_SIZE = 4

## Indexing

### Set embedding model

In [45]:
embeddings = OllamaEmbeddings(
    model=EMBEDDING_MODEL,
)

### Split texts and save them

In [47]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=OVERLAP,separators=["\n\n", "\n", " "])
documents_list = makeDocuments(path=DOCUMENTS_DIR,sample_size=SAMPLE_SIZE,verbose=VERBOSE)
all_splits = split_docs(text_splitter=text_splitter,documents_list=documents_list,verbose=VERBOSE)
save_documents(all_splits,output_dir=SPLITTED_DOCS_DIR,verbose=VERBOSE)


['1881-01-11.txt']
1881-01-11.txt
Splitting → 1881-01-11
Split blog post into 17 sub-documents.
Directory already exists


### Set if not exist a vector store

In [46]:
persist_dir = PERSISTENT_DIR
vector_store = make_db(persist_dir=PERSISTENT_DIR,verbose=VERBOSE,embeddings=embeddings)

Création du vector store...


### Embbed doc & store them in database once

In [48]:
def indexing():
    indexed_ids = vector_store.get(include=[])["ids"]
    if len(indexed_ids) == len(all_splits):
        print("Already indexed...")
    else :
        batch_size = BATCH_SIZE
        for i in tqdm(range(0, len(all_splits), batch_size), desc="Vectorizing",colour="cyan"):
            batch = all_splits[i:i+batch_size]
            vector_store.add_documents(batch)
indexing()

Vectorizing: 100%|██████████| 5/5 [00:01<00:00,  2.52it/s]


## Retrieval

In [35]:
def retrieve_context(query, k=2, separator="\n\n"):
    docs = vector_store.similarity_search_with_score(query=query, k=k)
    scores = []
    context = ""
    for doc, score in docs:
        scores.append(score)
        context += f"{doc.page_content}{separator}"  
    return context.strip(), scores, docs

In [38]:
query = """QUi est le tout premier député a avoir été appelé je veux un seul nom?"""
context, scores, docs = retrieve_context(query=query)

prompt = ChatPromptTemplate.from_messages([
    ("system", "Réponds à la question en utilisant le contexte suivant : {context}"),
    ("user", "Utilise ce contexte pour répondre à la question suivante : {question}")
])

model = OllamaLLM(model="llama3.2")

chain = prompt | model

response = chain.invoke({
    "question": query,
    "context": context
})
print(response)

H. Villain


In [14]:
print(docs)

[(Document(id='af779a89-113b-4389-a7a3-c339f520c186', metadata={'chunk_id': '1881-01-11_chunk_1', 'parent_source': '1881-01-11', 'source': '1881-01-11'}, page_content="tion de s'ajourner. (Bruyantes exclamations à droite.) Sur divers bancs à droite el à l'extrêmeoauchu.Non 1 non l Sur un grand nombre d'autres bancs. Mais si 1 c'est nécessaire 1 M. Georges Perin. Je demande la parole.\n\nM. Laroche-Joubert. Je demande lapa.\n\nrole.\n\nM le comte de Colbert-Laplace. Je demande la parole.\n\nM. le président. La parole est à M.\n\nPerin.\n\nM. Georges Perin. Messieurs, je viens, au nom d'un certain nombre de mes amis et en mon nom personnel, demander à la Chambre de repousser la proposition d'ajournement qui vient d'être faLe pac noire honorable président. (Très bien ! très bien ! à droite et à l'extrême gauche.) Autant qu'il m'a été permis de l'entendre au milieu du bruit, je crois que la seule raison que notre honorable président ait fait valoir 7our justifier sa proposition, c'est que 